In [2]:
import os
import json
import shutil

from library_dicom.dicom_processor.model.Series import Series
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.series import get_series_object


In [12]:
series_paths = get_series_path("/media/deeplearning/Elements/send_to_validated_dicom")
export_folder = '/media/deeplearning/Elements/json'

In [13]:
len(series_paths)

32

In [5]:
index_1 = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/Relevance/index_RELEVANCE.json'
index_2 = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/Relevance/index_part_2.json'
index_3 = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/Relevance/index_RELEVANCE_part3.json'

dataset_1 = []
with open(index_1) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_1.append(info)

dataset_2 = []
with open(index_2) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_2.append(info)

dataset_3 = []
with open(index_3) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset_3.append(info)


liste_index = dataset_1 + dataset_2 + dataset_3

treated = []
for serie_path in series_paths : 
    serie_uid = serie_path.split('/')[-1]
    if serie_uid in liste_index : 
        treated.append(serie_path)
print(len(treated))

0


In [10]:
for serie in treated : 
    series_paths.remove(serie)

In [14]:
not_image_folder=[]
non_valid_series=[]
non_attenuation_corrected=[]
empty_folders = []
secondary_series = []
localizer_series = []

for serie_path in series_paths:
    print(series_paths.index(serie_path))
    if(len(os.listdir(serie_path)) == 0):
        empty_folders.append(serie_path)
        continue
    try:
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        remove_bi_file(serie_path)
        if( dicom_serie.is_image_modality() == False):
            not_image_folder.append(serie_path)
        elif (dicom_serie.is_series_valid() == False):
            non_valid_series.append(serie_path)
        elif(type(dicom_serie) == SeriesPT and dicom_serie.is_corrected_attenuation() == False):
            non_attenuation_corrected.append(serie_path)
        elif(dicom_serie.is_localizer_series() == True):
            localizer_series.append(serie_path)
        else:

            write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], dicomsInfo)
            #print("json_file")
    except Exception as err:
        print(err)
        print(dicomsInfo)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [7]:
#Index of series we have treated in Relevance envoi 3
index = []
for serie_path in series_paths : 
    study_uid = serie_path.split('/')[-1]
    index.append(study_uid)
write_json_file('/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/', 'index_RELEVANCE_dernier', index)

In [8]:
#print(not_image_folder)
#print(non_valid_series)
#print(non_attenuation_corrected)
#print(empty_folders)
#print(secondary_series)
#print(localizer_series)

print(len(not_image_folder))
print(len(non_valid_series))
print(len(non_attenuation_corrected))
print(len(empty_folders))
print(len(secondary_series))
print(len(localizer_series))

0
0
30
0
0
3


In [9]:
for localizer_serie in localizer_series:
    shutil.rmtree(localizer_serie)

for not_image in not_image_folder:
    shutil.rmtree(not_image)

for non_attenuation in non_attenuation_corrected:
    shutil.rmtree(non_attenuation)

for non_valid_serie in non_valid_series : 
    shutil.rmtree(non_valid_serie)

for secondary_serie in secondary_series : 
    shutil.rmtree(secondary_serie)

In [10]:
for empty_folder in empty_folders:
    shutil.rmtree(empty_folder)

In [11]:
write_json_file("/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92", "wrong Series Relevance_dernier", json.dumps(non_valid_series))